# Need to fix the h-index. why average weighted link devided by nodes same question for betwennes

# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import community.community_louvain as community
import os, sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

In [2]:
network = 'lesmis'

#read edge list from csv file
edge_list = pd.read_csv('../Datasets/' + network + '.csv')
    
#create graph from the edge list
G = nx.from_pandas_edgelist(edge_list, edge_attr='weight', create_using=nx.Graph())

In [3]:
from Backbones import modularity_backbone as modv

unlabeled_edge_list = pd.read_csv('../Datasets/'+network+'.csv', sep=',')
g = nx.from_pandas_edgelist(unlabeled_edge_list, edge_attr='weight', create_using=nx.Graph())

k_nodes_to_remove = round(len(g.nodes())*0)
backbone, Q1, Q2, Q31, Q32 = modv.modularity_backbone(G, k_nodes_to_remove)

In [5]:
len(backbone.nodes())

77

# Zakaria


In [5]:
network = 'lesmis'

#read edge list from csv file
edge_list = pd.read_csv('../Datasets/' + network + '.csv')
    
#create graph from the edge list
G = nx.from_pandas_edgelist(edge_list, edge_attr='weight', create_using=nx.Graph())

In [11]:
'''
This module implements the disparity filter to compute a significance score of edge weights in networks
'''

import networkx as nx
import numpy as np
from scipy import integrate


def disparity_filter(G, weight='weight'):
    ''' Compute significance scores (alpha) for weighted edges in G as defined in Serrano et al. 2009
        Args
            G: Weighted NetworkX graph
        Returns
            Weighted graph with a significance score (alpha) assigned to each edge
        References
            M. A. Serrano et al. (2009) Extracting the Multiscale backbone of complex weighted networks. PNAS, 106:16, pp. 6483-6488.
    '''

    if nx.is_directed(G): #directed case
        N = nx.DiGraph()
        for u in G:

            k_out = G.out_degree(u)
            k_in = G.in_degree(u)

            if k_out > 1:
                sum_w_out = sum(np.absolute(G[u][v][weight]) for v in G.successors(u))
                for v in G.successors(u):
                    w = G[u][v][weight]
                    p_ij_out = float(np.absolute(w))/sum_w_out
                    alpha_ij_out = 1 - (k_out-1) * integrate.quad(lambda x: (1-x)**(k_out-2), 0, p_ij_out)[0]
                    N.add_edge(u, v, weight = w, alpha_out=float('%.4f' % alpha_ij_out))

            elif k_out == 1 and G.in_degree(G.successors(u)[0]) == 1:
                #we need to keep the connection as it is the only way to maintain the connectivity of the network
                v = G.successors(u)[0]
                w = G[u][v][weight]
                N.add_edge(u, v, weight = w, alpha_out=0., alpha_in=0.)
                #there is no need to do the same for the k_in, since the link is built already from the tail

            if k_in > 1:
                sum_w_in = sum(np.absolute(G[v][u][weight]) for v in G.predecessors(u))
                for v in G.predecessors(u):
                    w = G[v][u][weight]
                    p_ij_in = float(np.absolute(w))/sum_w_in
                    alpha_ij_in = 1 - (k_in-1) * integrate.quad(lambda x: (1-x)**(k_in-2), 0, p_ij_in)[0]
                    N.add_edge(v, u, weight = w, alpha_in=float('%.4f' % alpha_ij_in))
        return N

    else: #undirected case
        B = nx.Graph()
        for u in G:
            k = len(G[u])
            if k > 1:
                sum_w = sum(np.absolute(G[u][v][weight]) for v in G[u])
                for v in G[u]:
                    w = G[u][v][weight]
                    p_ij = float(np.absolute(w))/sum_w
                    alpha_ij = 1 - (k-1) * integrate.quad(lambda x: (1-x)**(k-2), 0, p_ij)[0]
                    B.add_edge(u, v, weight = w, alpha=float('%.4f' % alpha_ij))
        return B

def disparity_filter_alpha_cut(G,weight='weight',alpha_t=0.4, cut_mode='or'):
    ''' Performs a cut of the graph previously filtered through the disparity_filter function.

        Args
        ----
        G: Weighted NetworkX graph

        weight: string (default='weight')
            Key for edge data used as the edge weight w_ij.

        alpha_t: double (default='0.4')
            The threshold for the alpha parameter that is used to select the surviving edges.
            It has to be a number between 0 and 1.

        cut_mode: string (default='or')
            Possible strings: 'or', 'and'.
            It works only for directed graphs. It represents the logic operation to filter out edges
            that do not pass the threshold value, combining the alpha_in and alpha_out attributes
            resulting from the disparity_filter function.


        Returns
        -------
        B: Weighted NetworkX graph
            The resulting graph contains only edges that survived from the filtering with the alpha_t threshold

        References
        ---------
        .. M. A. Serrano et al. (2009) Extracting the Multiscale backbone of complex weighted networks. PNAS, 106:16, pp. 6483-6488.
    '''


    if nx.is_directed(G):#Directed case:
        B = nx.DiGraph()
        for u, v, w in G.edges(data=True):
            try:
                alpha_in =  w['alpha_in']
            except KeyError: #there is no alpha_in, so we assign 1. It will never pass the cut
                alpha_in = 1
            try:
                alpha_out =  w['alpha_out']
            except KeyError: #there is no alpha_out, so we assign 1. It will never pass the cut
                alpha_out = 1

            if cut_mode == 'or':
                if alpha_in<alpha_t or alpha_out<alpha_t:
                    B.add_edge(u,v, weight=w[weight])
            elif cut_mode == 'and':
                if alpha_in<alpha_t and alpha_out<alpha_t:
                    B.add_edge(u,v, weight=w[weight])
        return B

    else:
        B = nx.Graph()#Undirected case:
        for u, v, w in G.edges(data=True):

            try:
                alpha = w['alpha']
            except KeyError: #there is no alpha, so we assign 1. It will never pass the cut
                alpha = 1

            if alpha<alpha_t:
                B.add_edge(u,v, weight=w[weight])
        return B

if __name__ == '__main__':

    #from data_karate import *
    #from data_Lesmiserables import *
    #from data_dolphin import *
    #from data_got import *
    #from data_football import *
    #from data_egofacebook import *
    #from data_grqc import *
    #from data_HepTh import *
    #from data_CondMat import *
    #from data_enronemail import *

    ###################### Karate club ##################################
    #from data_karate import *
    ######################################################################
    ###################### Les Miserables ##################################
    #from data_Lesmiserables import *
    ######################################################################
    ###################### Daulphins #####################################
    #from data_dolphin import *
    ########################################################################
    ###################### Game of thrones ################################
    #from data_got import *
    ######################################################################
    ###################### football ######################################
    #from data_football import *
    ######################################################################
    #####################################################################
    #from new_data.terrorist import *
    ################################################################################
    #####################################################################
    #from new_data.ecologie import *
    ################################################################################
    #####################################################################
    #from new_data.adjnoun import *
    ################################################################################
    ############################## euroroads #######################################
    from new_data.euroroads import *
    ################################################################################
    ############################## crime #######################################
    #from new_data.crime import *
    ################################################################################

    #G = nx.barabasi_albert_graph(1000, 5)
    #G=nx.karate_club_graph()
    G=g

    for u, v in G.edges():
        G[u][v]['weight'] = np.random.randint(1,100)
    alpha = 0.2
    G = disparity_filter(G)
    G2 = nx.Graph([(u, v, d) for u, v, d in G.edges(data=True) if d['alpha'] < alpha])
    print ('alpha = %s' % alpha)
    print ('original: nodes = %s, edges = %s' % (G.number_of_nodes(), G.number_of_edges()))
    print ('backbone: nodes = %s, edges = %s' % (G2.number_of_nodes(), G2.number_of_edges()))
    print (G2.edges(data=True))
    print(G2.nodes())


ModuleNotFoundError: No module named 'new_data'

In [12]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from scipy import integrate
from backbone import *
from methods_overlapping import *
from sklearn.metrics import jaccard_score
from slpa import *
import math


#Networks:
#  -------

#g=nx.read_pajek('karate_club_weighted.net')

g=nx.read_weighted_edgelist(r'C:\Users\HP\Desktop\Programme1\Backbone2\weightedNets\karate_club_weighted.dat')
network='weightedNets/karate_club_weighted.dat'

#g = nx.read_gml(r'C:\Users\HP\Desktop\Programme1\Backbone2\weightedNets\lesmis.gml')
#network='weightedNets/lesmis.gml'

#g=nx.read_weighted_edgelist('weightedNets/got.dat')
#network='weightedNets/got.dat'

#g=nx.read_weighted_edgelist('weightedNets/train.dat')
#network='weightedNets/train.dat'

#g=nx.read_weighted_edgelist('toy.dat')
#g=nx.read_weighted_edgelist('USairoport.dat')

#g=nx.read_weighted_edgelist('weightedNets/Caenorhabditis.dat')
#g=nx.read_weighted_edgelist('weightedNets/Davis_southern_club_women.dat')
#g=nx.read_weighted_edgelist('weightedNets/Facebook-likeForum.dat')
#g=nx.read_weighted_edgelist('weightedNets/Facebook-likeSocial.dat')
#g=nx.read_weighted_edgelist('weightedNets/FreemansEIES.dat')
#network='weightedNets/FreemansEIES.dat'

#g=nx.read_weighted_edgelist('weightedNets/Intra-organisational.dat')
#network='weightedNets/Intra-organisational.dat'

#g=nx.read_weighted_edgelist('weightedNets/scientificCollaboration.dat')
#g=nx.read_weighted_edgelist('weightedNets/USpowergrid.dat')
#network='weightedNets/USpowergrid.dat'

#g=nx.read_weighted_edgelist('weightedNets/Davis_southern_club_women2.dat')
#g=nx.read_weighted_edgelist('weightedNets/Intra-organisational2.dat')
#g=nx.read_weighted_edgelist('weightedNets/Intra-organisational3.dat')
#g=nx.read_weighted_edgelist('weightedNets/Intra-organisational4.dat')
#g=nx.read_weighted_edgelist('weightedNets/scientificCollaboration2.dat')

#  Alpha :
#  -----
alpha = 0.9

#  mu :
#  -----
mu = 0.5 #tuning parameter for degree centrality

s=0.3

ep=6.0
ITcommunityfinder=1
width=12
hight=12
w=12
h=12

width_o=15
hight_o=15



#colors=['Green', 'Red', 'blue', '#CD853F', '#87CEEB', 'Maroon', 'orange', 'purple', 'Yellow', 'Gray']
colors=['Green', 'Maroon', 'Red', 'blue', 'orange', 'purple', '#87CEEB', '#CD853F', 'Gray', 'Yellow']
#colors=['Red', 'Green', 'blue', '#CD853F', '#87CEEB', 'Maroon', 'orange', 'purple', 'Yellow', 'Gray']

########################### The graph #####################################""
l_nodes=g.nodes() #list of nodes
dict_graph=dict()
for i in g:
    dict_graph[i]=[]
    for j in g[i]:
        dict_graph[i].append(j)
print('dict_graph=',dict_graph) #dict(node: neighbors)



############ dict_tri= node: degree sorted #####################################
list_edges=g.edges(data=True) # list: list_edges=[(original node, destination node, weight)]
print('list_edges=',list_edges)

dict_strength=dict() #dict(node: its total weight)
for i in l_nodes:
    dict_strength[i]=0
    for e in list_edges:
        if (e[0]==i) or (e[1]==i):
            dict_strength[i]=dict_strength[i]+e[2]['weight']
            #dict_strength[i]=dict_strength[i]+e[2]['value']
print('dict_strength=',dict_strength)

d_degree=nx.degree_centrality(g) #d_degree=dict(node:degree)
#print('d_degree=',d_degree)

#mu=0.5  #tuning parameter
mu1=1-mu
dict_degree=dict()
for i in l_nodes:
    dict_degree[i]=(math.pow(d_degree[i],mu1))*(math.pow(dict_strength[i],mu))

# le tri
d = dict_degree
l = []
for i in d:
    l.append (i)
n = len (d)

for i in range (0, n - 1):
    max = i
    for j in range (i + 1, n):
        if d[l[max]] < d[l[j]]:
            max = j
    if max != i:
        x = l[i]
        l[i] = l[max]
        l[max] = x
dict_tri = dict ()
for i in l:
    dict_tri[i] = d[i]
#"""
dict_degree=dict_tri
#print('List of nodes sorted by degree:',l)
print('dict_tri=',dict_tri)
print('dict_degree=',dict_tri)
#"""




#"""
############### détecter les communautés avec SLPA #############################
c = find_communities(g,ITcommunityfinder,0.01)  # compute communities
#la liste des partition:
i=0
keysp=list()
for j in c:
    keysp.append(i)
    i=i+1
#print(keysp)

# calculer dictionnaire partition = partition:list of nodes
partition=dict()
for i in keysp:
    partition[i]=[]
i=0
for j in c:
    for k in c[j]:
        partition[i].append(k)
    i=i+1
print('partition_nodes=',partition)
print('partition=',partition)
print('nc=',len(partition))
d_community=partition
#"""
############ list_over=list of overlapping nodes ###############################
lo,membership,do=list_overlapping(g,d_community)
print('membership=',membership) #node:membership
print('lo= ',lo)
#"""
############ dict_over=Overlapping node:list of its neighbors ##################
dict_over=dict()
for i in lo:
    dict_over[i]=dict_graph[i]
############ Neighbor_over_tri=list of the neighbors of all the overlapping nodes sorted #
Neighbor_over=list()
ln=list()
for i in lo:
    for j in dict_graph[i]:
        ln.append(j)

Neighbor_over=list(set(ln))

d=dict()
for i in Neighbor_over:
    d[i]=dict_tri[i]
n = len (d)
for i in range (0, n - 1):
    max = i
    for j in range (i + 1, n):
        if d[Neighbor_over[max]] < d[Neighbor_over[j]]:
            max = j
    if max != i:
        x = Neighbor_over[i]
        Neighbor_over[i] = Neighbor_over[max]
        Neighbor_over[max] = x
Neighbor_over_tri=dict()
for i in Neighbor_over:
    Neighbor_over_tri[i] = dict_tri[i]

print('Neighbor_over_tri= ',Neighbor_over_tri)
#print('ego=',len(list(Neighbor_over_tri))/len(list(dict_graph)))
#"""






############################# Original network ####################################
###################  node_community= node: list of its partitions #######################
partition2=dict()
for i in dict_graph:
    partition2[i]=[]
for i in dict_graph:
    for j in partition:
        if (partition[j].count(i)!=0):
            partition2[i].append(j)
node_community=partition2 #dict(node: list of its partitions)
################ partition_color= partition: color #############################
partition_color=dict()
k=0
for i in partition:
    partition_color[i]=colors[k]
    k=k+1
################ node_color= node:color ########################################
node_color=dict()
for i in g:
    n=node_community[i]
    node_color[i]=partition_color[n[0]]

list_colors=[]
for i in g:
    if lo.count(i)==1:
        list_colors.append('#F5F5F5')
    else:
        list_colors.append(node_color[i])


############### Draw the original network ###############################################
d=dict_degree

#g=g_inter
d2=dict()
for i in g:
    d2[i]=d[i]


plt.subplots_adjust(left = 0.01, bottom = 0.01, right = 0.99, top = 0.99)
plt.figure(figsize=(width_o,hight_o))  # image is 8 x 8 inches
plt.axis('off')

# edges
all_weights = []
for (node1,node2,data) in g.edges(data=True):
    #print('node1,node2,data: ',data)
    all_weights.append(data['weight']) #we'll use this when determining edge thickness

#4 b. Get unique weights
unique_weights = list(set(all_weights))
#pos=nx.circular_layout(g)
pos=nx.spring_layout(g)
#4 c. Plot the edges - one by one!
node_list=g.nodes()
for weight in unique_weights:
    weighted_edges = [(node1,node2) for (node1,node2,edge_attr) in g.edges(data=True) if edge_attr['weight']==weight]
    width = weight*len(node_list)*ep/sum(all_weights)
    nx.draw_networkx_edges(g,pos,edgelist=weighted_edges,width=width)
a=len(node_list)*ep
b=sum(all_weights)

nx.draw_networkx(g, pos, with_labels=True, node_size=[v * 80 for v in d2.values()] , node_color=list_colors, font_size=12, font_color='k', font_family='sans-serif', font_weight='bold')

plt.savefig('0OriginalNetwork.png',dpi=300)













############################# Global Component ####################################
#print('1',g.edges(data=True))
# Le graph(graph_inter)+dictionnaire(g_inter) intra des noeuds overlapping et non-overlapping
g_inter=globalComponent(g,partition)
#print('global component dictionary: ',g_inter)
#print(nx.info(graph_inter))
ss1=0
ss2=0
for e in g.edges(data=True):
    ss1=ss1+e[2]['weight']
for e in g_inter.edges(data=True):
    ss2=ss2+e[2]['weight']
print('Mixing parameter weighted: ',ss2/ss1)
print('Mixing parameter: ',len(list(g_inter.edges(data=True)))/len(list(g.edges(data=True))))
print('Global component: ',g_inter.edges(data=True))

################### Drawing the Global Component ###############################
g1=g_inter
###################  node_community= node: list of its partitions #######################
partition2=dict()
for i in dict_graph:
    partition2[i]=[]
for i in dict_graph:
    for j in partition:
        if (partition[j].count(i)!=0):
            partition2[i].append(j)
#print('partition2=',partition2)
node_community=partition2 #dict(node: list of its partitions)
################ partition_color= partition: color #############################
partition_color=dict()
k=0
for i in partition:
    partition_color[i]=colors[k]
    k=k+1
################ node_color= node:color ########################################
node_color=dict()

for i in g1:
    n=node_community[i]
    node_color[i]=partition_color[n[0]]

list_colors=[]

for i in g1:
    if lo.count(i)==1:
        list_colors.append('#F5F5F5')
    else:
        list_colors.append(node_color[i])



############### Draw the figure of the global component ########################
d=dict_degree

#g1=g_inter
d2=dict()
for i in g1:
    d2[i]=d[i]

plt.subplots_adjust(left = 0.01, bottom = 0.01, right = 0.99, top = 0.99)
plt.figure(figsize=(w, h))  # image is 8 x 8 inches
plt.axis('off')

# edges
all_weights = []
for (node1,node2,data) in g1.edges(data=True):
    #print('node1,node2,data: ',data)
    all_weights.append(data['weight']) #we'll use this when determining edge thickness

#4 b. Get unique weights
unique_weights = list(set(all_weights))
pos=nx.circular_layout(g1)
#pos=nx.spring_layout(g1)
#4 c. Plot the edges - one by one!
node_list=g1.nodes()
for weight in unique_weights:
    weighted_edges = [(node1,node2) for (node1,node2,edge_attr) in g1.edges(data=True) if edge_attr['weight']==weight]
    width = weight*len(node_list)*ep/sum(all_weights)
    nx.draw_networkx_edges(g1,pos,edgelist=weighted_edges,width=width)
a=len(node_list)*ep
b=sum(all_weights)

nx.draw_networkx(g1, pos, with_labels=True, node_size=[v * 80 for v in d2.values()] , node_color=list_colors, font_size=12, font_color='k', font_family='sans-serif', font_weight='bold')

plt.savefig('1globalComponent.png',dpi=300)

















############ Extract the Serrano backbone of the Global component ###############
for u, v in g1.edges():
    g1[u][v]['weight'] = np.random.randint(1,100)

# Filter and new serrano backbone
g1 = disparity_filter(g1)
g2 = nx.Graph([(u, v, d) for u, v, d in g1.edges(data=True) if d['alpha'] < alpha])
print ('alpha = %s' % alpha)
#print ('original: nodes = %s, edges = %s' % (g.number_of_nodes(), g.number_of_edges()))
print ('Serrano backbone of the GC: nodes = %s, edges = %s' % (g2.number_of_nodes(), g2.number_of_edges()))
print ('Edges of serrano GC: ',g2.edges(data=True))
print('Nodes of serrano GC: ',g2.nodes())
l_edgesSerrano=list(g2.edges(data=True))



################## Drawing Serrano backbone of the Global Component ############
e=g2.edges(data=True)
g=nx.Graph()
ee=[(i[0],i[1]) for i in g.edges(data=True)]
for i in e:
    ee.append((i[0],i[1]))

g1=nx.read_weighted_edgelist(network)
dict_weights=dict()
for i in g1.edges(data=True):
    dict_weights[(i[0],i[1])]=i[2]['weight']

e2=[]
for i in ee:
    if list(dict_weights).count((i[0],i[1]))!=0:
        e2.append((i[0],i[1],dict_weights[(i[0],i[1])]))
    else:
        e2.append((i[0],i[1],dict_weights[(i[1],i[0])]))
#print(ee)
#print(e2)
g.add_weighted_edges_from(e2)

d=dict_degree

d2=dict()
for i in g:
    d2[i]=d[i]



G=g
#print('huuu',G.edges(data=True))
elarge = [(u, v) for (u, v, d) in G.edges(data=True) if d['weight'] > 20]
esmall = [(u, v) for (u, v, d) in G.edges(data=True) if d['weight'] <= 20]


plt.subplots_adjust(left = 0.01, bottom = 0.01, right = 0.99, top = 0.99)
plt.figure(figsize=(w, h))  # image is 8 x 8 inches

#pos = nx.spring_layout(G)  # positions for all nodes
#pos=nx.shell_layout(G)
pos=nx.circular_layout(G)

################ node_color= node:color ########################################
node_color=dict()

for i in G:
    n=node_community[i]
    node_color[i]=partition_color[n[0]]

list_colors=[]

for i in G:
    if lo.count(i)==1:
        list_colors.append('#F5F5F5')
    else:
        list_colors.append(node_color[i])

# nodes
nx.draw_networkx_nodes(G, pos, node_size=[v * 80 for v in d2.values()], node_color=list_colors)

# edges
all_weights = []
#4 a. Iterate through the graph nodes to gather all the weights
for (node1,node2,data) in G.edges(data=True):
    all_weights.append(data['weight']) #we'll use this when determining edge thickness




#print(g2.edges(data=True))
node_list2=g2.nodes()
all_weights2 = []
s2=0
for (node1,node2,data) in g2.edges(data=True):
    all_weights2.append(data['weight']) #we'll use this when determining edge thickness
    s2=s2+data['weight']
#4 b. Get unique weights
unique_weights = list(set(all_weights))

#4 c. Plot the edges - one by one!
node_list=G.nodes()
for i in G.edges(data=True):
    if list(dict_weights).count((i[0],i[1]))!=0:
        weight=dict_weights[(i[0],i[1])]
    else:
        weight=dict_weights[(i[1],i[0])]
    #4 d. Form a filtered list with just the weight you want to draw
    weighted_edges = [(node1,node2) for (node1,node2,edge_attr) in G.edges(data=True) if edge_attr['weight']==weight]
    #4 e. I think multiplying by [num_nodes/sum(all_weights)] makes the graphs edges look cleaner
    width = weight*a/b
    nx.draw_networkx_edges(G,pos,edgelist=weighted_edges,width=width)


# labels
nx.draw_networkx_labels(G, pos, node_size=[v * 80 for v in d2.values()], node_color=list_colors, font_size=12, font_color='k', font_family='sans-serif', font_weight='bold')

plt.axis('off')

plt.savefig('2serrano1.png',dpi=300)


















########################## Overlap-Hub backbone ##############################
g = nx.read_weighted_edgelist(network)
l_nodes=g.nodes() #list of nodes
dict_graph=dict()
for i in g:
    dict_graph[i]=[]
    for j in g[i]:
        dict_graph[i].append(j)

networksize=int(len(list(g.nodes()))*s)

# extracting the list of hubs
size=len(Neighbor_over_tri)
l_hubs=list(dict_degree)
l_hubs=list_extract2(l_hubs,size)

# ego network
l_ego=list(set(l_hubs+lo))
dict_ego=dict()
for i in l_ego:
    dict_ego[i]=dict_tri[i]
d = dict_ego
l = []
for i in d:
    l.append (i)
n = len (d)

for i in range (0, n - 1):
    max = i
    for j in range (i + 1, n):
        if d[l[max]] < d[l[j]]:
            max = j
    if max != i:
        x = l[i]
        l[i] = l[max]
        l[max] = x
dict_tri = dict ()
for i in l:
    dict_tri[i] = d[i]
dict_ego=dict_tri

dict_ego=dict_extract(dict_ego,networksize)

l_nodes=list(dict_graph)

dict_graph=immunize2(dict_graph,dict_ego,l_nodes)
#print('dict_graph',dict_graph)


g_OH=nx.Graph()
l_edges=[]
for i in dict_graph:
    for j in dict_graph[i]:
        l_edges.append((i,j))

for li in l_edges:
    l_edges.remove((li[1],li[0]))
#print('l_edges=',l_edges)

g = nx.read_weighted_edgelist(network)
#print('lalala',g.edges(data=True))
l_edges2=[]
edges_sorted=dict()
for e in l_edges:
    #print(e)
    for v in g.edges(data=True):
        #print(node1,node2,data)
        if (e[0]==v[0] and e[1]==v[1]) or (e[0]==v[1] and e[1]==v[0]):
            #l_edges2.append((e[0],e[1],data['weight']))
            l_edges2.append((e[0],e[1],v[2]['weight']))
            edges_sorted[(e[0],e[1])]=v[2]['weight']

g_OH.add_weighted_edges_from(l_edges2)



d = edges_sorted
l = []
for i in d:
    l.append (i)
n = len (d)

for i in range (0, n - 1):
    max = i
    for j in range (i + 1, n):
        if d[l[max]] > d[l[j]]:
            max = j
    if max != i:
        x = l[i]
        l[i] = l[max]
        l[max] = x
dict_t = dict ()
for i in l:
    dict_t[i] = d[i]
edges_sorted=dict_t


l_lcc=list()
for i in edges_sorted:
    x=i
    g_OH.remove_edges_from([i])
    l_lcc=list(nx.connected_components(g_OH))
    if (len(l_lcc)>1):
        g_OH.add_weighted_edges_from([(x[0],x[1],edges_sorted[(x[0],x[1])])])
        continue
l_edgesOH=list(g_OH.edges(data=True))
print('l_edgesOH=',list(g_OH.edges(data=True)))
print('size OH: ',len(list(g_OH.nodes())))

################ node_color= node:color ########################################
node_color=dict()

for i in g_OH:
    n=node_community[i]
    node_color[i]=partition_color[n[0]]

list_colors=[]

for i in g_OH:
    if lo.count(i)==1:
        list_colors.append('#F5F5F5')
    else:
        list_colors.append(node_color[i])



############### Draw the figure of the global component ########################
d=dict_degree

#g1=g_inter
d2=dict()
for i in g_OH:
    d2[i]=d[i]


plt.subplots_adjust(left = 0.01, bottom = 0.01, right = 0.99, top = 0.99)
plt.figure(figsize=(w, h))  # image is 8 x 8 inches
plt.axis('off')

# edges
all_weights = []
for (node1,node2,data) in g_OH.edges(data=True):
    #print('node1,node2,data: ',data)
    all_weights.append(data['weight']) #we'll use this when determining edge thickness

#4 b. Get unique weights
unique_weights = list(set(all_weights))
pos=nx.circular_layout(g_OH)
#pos=nx.spring_layout(g1)
#4 c. Plot the edges - one by one!
node_list=g_OH.nodes()
for weight in unique_weights:
    weighted_edges = [(node1,node2) for (node1,node2,edge_attr) in g_OH.edges(data=True) if edge_attr['weight']==weight]
    width = weight*len(node_list)*ep/sum(all_weights)
    nx.draw_networkx_edges(g_OH,pos,edgelist=weighted_edges,width=width)
a=len(node_list)*ep
b=sum(all_weights)

nx.draw_networkx(g_OH, pos, with_labels=True, node_size=[v * 80 for v in d2.values()] , node_color=list_colors, font_size=12, font_color='k', font_family='sans-serif', font_weight='bold')

plt.savefig('4OH.png',dpi=300)











################################################################################
l_edgesInter=[]
for e1 in l_edgesSerrano:
    for e2 in l_edgesOH:
        if ((e1[0]==e2[0]) and (e1[1]==e2[1])) or ((e1[0]==e2[1]) and (e1[1]==e2[0])):
            for v in g.edges(data=True):
                if (e1[0]==v[0] and e1[1]==v[1]) or (e1[0]==v[1] and e1[1]==v[0]):
                    l_edgesInter.append((e1[0],e1[1],v[2]['weight']))
            #l_edgesInter.append(e1)
print('l_edgesInter: ',l_edgesInter)
g_Inter=nx.Graph()
g_Inter.add_weighted_edges_from(l_edgesInter)
l_inter=list(g_Inter.nodes())
print('edges Inter: ',list(g_Inter.edges(data=True)))

print('Nodes inter: ',set(list(g_OH.nodes()))&set(list(G.nodes())))
l_int=list(set(list(g_OH.nodes()))&set(list(G.nodes())))
l_del=list(set(list(G.nodes())))
for i in l_int:
    l_del.remove(i)

size_serr=len(l_del)
networksize=networksize-size_serr


edges_serrano=list(G.edges(data=True))
#print('edges_serrano',edges_serrano)
for e in list(g_Inter.edges(data=True)):
    edges_serrano.remove(e)

################################################################################



# ego network


dict_ego=dict_extract(dict_ego,networksize)
l_nodes=list(dict_graph)

dict_graph=immunize2(dict_graph,dict_ego,l_nodes)
#print('dict_graph',dict_graph)


g_OH2=nx.Graph()
l_edges=[]
for i in dict_graph:
    for j in dict_graph[i]:
        l_edges.append((i,j))

for li in l_edges:
    l_edges.remove((li[1],li[0]))
print('l_edges=',l_edges)



l_edges2=[]
edges_sorted=dict()
for e in l_edges:
    #print(e)
    for v in g.edges(data=True):
        #print(node1,node2,data)
        if (e[0]==v[0] and e[1]==v[1]) or (e[0]==v[1] and e[1]==v[0]):
            #l_edges2.append((e[0],e[1],data['weight']))
            l_edges2.append((e[0],e[1],v[2]['weight']))
            edges_sorted[(e[0],e[1])]=v[2]['weight']

g_OH2.add_weighted_edges_from(l_edges2)



d = edges_sorted
l = []
for i in d:
    l.append (i)
n = len (d)

for i in range (0, n - 1):
    max = i
    for j in range (i + 1, n):
        if d[l[max]] > d[l[j]]:
            max = j
    if max != i:
        x = l[i]
        l[i] = l[max]
        l[max] = x
dict_t = dict ()
for i in l:
    dict_t[i] = d[i]
edges_sorted=dict_t


l_lcc=list()
for i in edges_sorted:
    x=i
    g_OH2.remove_edges_from([i])
    l_lcc=list(nx.connected_components(g_OH2))
    if (len(l_lcc)>1):
        g_OH2.add_weighted_edges_from([(x[0],x[1],edges_sorted[(x[0],x[1])])])
        continue
l_edgesOH2=list(g_OH2.edges(data=True))
print('l_edgesOH2=',list(g_OH2.edges(data=True)))

l_edgesF=l_edgesOH2+edges_serrano

l_edgesFinal=[]
for e in l_edgesF:
    l_edgesFinal.append((e[0],e[1],e[2]['weight']))

print('l_edgesFinal: ',l_edgesF)
g_VF=nx.Graph()
g_VF.add_weighted_edges_from(l_edgesFinal)



################ node_color= node:color ########################################
node_color=dict()

for i in g_VF:
    n=node_community[i]
    node_color[i]=partition_color[n[0]]

list_colors=[]

for i in g_VF:
    if lo.count(i)==1:
        list_colors.append('#F5F5F5')
    else:
        list_colors.append(node_color[i])



############### Draw the figure of the global component ########################
d=dict_degree

#g1=g_inter
d2=dict()
for i in g_VF:
    d2[i]=d[i]

print('g_VF',g_VF.edges(data=True))
plt.subplots_adjust(left = 0.01, bottom = 0.01, right = 0.99, top = 0.99)
plt.figure(figsize=(w, h))  # image is 8 x 8 inches
plt.axis('off')

# edges
all_weights = []
for (node1,node2,data) in g_VF.edges(data=True):
    #print('node1,node2,data: ',data)
    all_weights.append(data['weight']) #we'll use this when determining edge thickness

#4 b. Get unique weights
unique_weights = list(set(all_weights))
pos=nx.circular_layout(g_VF)
#pos=nx.spring_layout(g1)
#4 c. Plot the edges - one by one!
node_list=g_VF.nodes()
for weight in unique_weights:
    weighted_edges = [(node1,node2) for (node1,node2,edge_attr) in g_VF.edges(data=True) if edge_attr['weight']==weight]
    width = weight*len(node_list)*ep/sum(all_weights)
    nx.draw_networkx_edges(g_VF,pos,edgelist=weighted_edges,width=width)
a=len(node_list)*ep
b=sum(all_weights)

nx.draw_networkx(g_VF, pos, with_labels=True, node_size=[v * 80 for v in d2.values()] , node_color=list_colors, font_size=12, font_color='k', font_family='sans-serif', font_weight='bold')

plt.savefig('3VF.png',dpi=300)


ModuleNotFoundError: No module named 'backbone'